In [1]:
%pip install openai transformers torch sentencepiece
%pip install openai==0.28


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade tokenizers
%pip install tokenizers==0.13.3


  Obtaining dependency information for tokenizers from https://files.pythonhosted.org/packages/ae/1a/4526797f3719b0287853f12c5ad563a9be09d446c44ac784cdd7c50f76ab/tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl.metadata
  Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl (2.7 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.32.1 requires tokenizers!=0.11.3,<0.14,>=0.11.1, but you have tokenizers 0.21.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for tokenizers==0.13.3 from https://files.pythonhosted.org/

In [3]:
from transformers import pipeline

summarizer_extractive = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_extractive(text, max_length=150):
    summary = summarizer_extractive(text, max_length=max_length, min_length=50, do_sample=False)
    return summary[0]['summary_text']

# Example usage:
with open("article.txt", "r") as file:
    text = file.read()

print("Extractive Summary:\n", summarize_extractive(text))

/Users/saideepthimakineni/anaconda3/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
/Users/saideepthimakineni/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/Users/saideepthimakineni/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Extractive Summary:
 Emma always dreamed of seeing the world beyond her small coastal village. Her father, a fisherman, told tales of distant lands filled with vibrant markets, bustling streets, and people speaking languages she'd never heard. With courage in her heart and a map drawn from her father's stories, she sailed out, leaving the comfort of familiarity behind.


In [4]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

def summarize_abstractive_openai(text, max_tokens=150):
    prompt = f"Summarize this article concisely:\n\n{text}\n\nSummary:"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

# Example usage:
with open("article.txt", "r") as file:
    text = file.read()

print("OpenAI Abstractive Summary:\n", summarize_abstractive_openai(text))

OpenAI Abstractive Summary:
 Emma, a girl from a small coastal village, dreams of exploring the world beyond. She discovers an old boat, repairs it, and sets sail, following her father's stories. After facing challenges at sea, she arrives at a bustling city, ready for adventure.


In [5]:
import os
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()
API_TOKEN = os.getenv("HF_API_TOKEN")

login(API_TOKEN)

In [6]:
import os
from dotenv import load_dotenv
import requests

load_dotenv()

API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
headers = {"Authorization": f"Bearer {os.getenv('HF_API_TOKEN')}"}

def summarize_mistral_api(article, max_tokens=150):
    prompt = f"<s>[INST] Summarize this article concisely:\n\n{article}\n[/INST]"
    response = requests.post(
        API_URL, headers=headers, json={
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": max_tokens,
                "return_full_text": False,
            },
            "options": {"wait_for_model": True}
        }
    )
    response.raise_for_status()
    return response.json()[0]['generated_text'].strip()

with open("article.txt", "r") as file:
    text = file.read()

summary = summarize_mistral_api(text)
print("Mistral Summary:\n", summary)

Mistral Summary:
 Emma, a young woman from a small coastal village, dreamed of traveling the world beyond her home. Inspired by her father's tales of vibrant markets and bustling streets, she discovered an abandoned boat and spent weeks repairing it. With determination and the map from her father's stories, she set sail, facing challenges and leaving her comfort zone behind. After weeks at sea, she reached a bustling city, fulfilling her dreams and ready for new adventures.


In [7]:
def combined_summary(text):
    extracted_summary = summarize_extractive(text, max_length=250)
    abstractive_summary = summarize_abstractive_openai(extracted_summary, max_tokens=150)
    return abstractive_summary

# Example usage:
print("Combined Summary:\n", combined_summary(text))

Your max_length is set to 250, but your input_length is only 218. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=109)


Combined Summary:
 Emma leaves her small coastal village to explore distant lands after being inspired by her father's tales of vibrant markets and bustling streets.


In [8]:
%pip install evaluate rouge-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [9]:
from rouge_score import rouge_scorer

reference_summary = "Emma dreamed of exploring beyond her village, repaired an old boat, sailed through challenges, and finally reached an exciting new city, fulfilling her adventurous dreams."

generated_summary = summary  # Your Mistral generated summary from your previous code.

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, generated_summary)

print(f"ROUGE-1 Precision: {scores['rouge1'].precision:.2f}")
print(f"ROUGE-1 Recall: {scores['rouge1'].recall:.2f}")
print(f"ROUGE-1 F1 Score: {scores['rouge1'].fmeasure:.2f}\n")

print(f"ROUGE-L Precision: {scores['rougeL'].precision:.2f}")
print(f"ROUGE-L Recall: {scores['rougeL'].recall:.2f}")
print(f"ROUGE-L F1 Score: {scores['rougeL'].fmeasure:.2f}")

ROUGE-1 Precision: 0.25
ROUGE-1 Recall: 0.76
ROUGE-1 F1 Score: 0.37

ROUGE-L Precision: 0.19
ROUGE-L Recall: 0.60
ROUGE-L F1 Score: 0.29
